Creating a matrix product operator using singular value decompositions

Siddhartha Harmalkar

June 2020

# Creation
Algorithm: Given an operator $\rho_{i,j}$ with indices $i,j\in\mathbb Z_d^n$:
(<mark>todo: update max bond dimension</mark>)
1. Reshape: $\rho_{i,j}= P_{(i_1\cdots i_n),(j_1\cdots j_n)}$ has dimensions $d^n\times d^n$
2. Permute: $P_{(i_1\cdots i_n),(j_1\cdots j_n)}= T_{(i_1j_1),\cdots ,(i_nj_n)}$ has dimensions $d^2\times\cdots \times d^2$ ($n$ times)
3. Collect: $T_{(i_1j_1),(i_2j_2\cdots i_nj_n)}$ has dimensions $d^2\times d^{2(n-1)}$
4. First site:
	- SVD: $T_{(i_1j_1),(i_2j_2\cdots i_nj_n)}=\sum_{\alpha\in \mathbb Z_{D_1}} U_{(i_1j_1),\alpha}S_{\alpha,\alpha} (V^\dagger)_{\alpha,(i_2j_2\cdots i_nj_n)}$, where $D_1\in \{1,\cdots,d^2\}$ is determined by truncation 
	- Save: $M^{(1)}_{\alpha,u,\beta}=\delta_{\alpha,1}U_{u,\beta}$ has dimensions $1\times d^2\times D_1$
	- Update: $T^{(1)}_{(\alpha i_2j_2),(i_3j_3\cdots i_nj_n)}=S_{\alpha,\alpha}(V^\dagger)_{\alpha,(i_2j_2\cdots i_nj_n)}$ has dimensions $D_1 d^2\times d^{2(n-2)}$
5. Inner sites: For $k\in \{2,\cdots,n-1\}$:
	- SVD: $T^{(k)}_{(\alpha i_kj_k),(i_{k+1}j_{k+1}\cdots i_nj_n)}=\sum_{\beta\in \mathbb Z_{D_k}} U_{(\alpha i_kj_k),\beta}S_{\beta,\beta} (V^\dagger)_{\beta,(i_{k+1}j_{k+1}\cdots i_nj_n)}$, where $D_k\in\{1,\cdots,D_{k-1}d^2\}$
	- Save: $M^{(k)}_{\alpha,u,\beta}=U_{(\alpha u),\beta}$ has dimensions $D_{k-1}\times d^2\times D_k$
	- Update: $T_{(\alpha i_{k+1}j_{k+1}),(i_{k+2}j_{k+2}\cdots i_nj_n)}=S_{\alpha,\alpha} (V^\dagger)_{\alpha,(i_{k+1}j_{k+1}\cdots i_nj_n)}$ has dimensions $D_k d^2\times d^{2(n-k-1)}$
6. Last site:
	- Save: $M^{(n)}_{\alpha,u,\beta}=T^{(n-1)}_{(\alpha,u),\beta}$ has dimensions $D_{n-1}\times d^2\times 1$ (note that $T^{(n-1)}_{(\alpha i_nj_n),\beta}$ has dimensions $D_{n-1} d^2\times 1$)
7. Return $\{M^{(k)}\}_{k\in\mathbb Z_n}$

In [85]:
import numpy as np
from numpy import linalg as la

# Given a density matrix, return its matrix product operator representation
# Indices are stored in the following format: (bond,phys,bond)
def state_to_mpo(state, n, d, verbose=False, **kwargs):
    # If not given a maximum bond dimension, set it to the maximum possible - d^(4n) (TODO)?
    #TODO: add a truncation magnitude cuttoff instead of just max_bd
    max_bd = kwargs.get('max_bd', d**(4*n))
    
    # Constructing environment tensor                                                                              
    tensor_shape = tuple([d]*(2*n))
    tensor = state.reshape(tensor_shape)
    tensor_axes = [i for i in range(2*n)]
    T_axes = [int(i/2) if i%2 == 0 else int(n + (i/2)) for i in range(2*n)]
    d2 = d**2
    T = np.moveaxis(tensor, tensor_axes, T_axes)
    T = T.reshape((d2,d2**(n-1)))
    if verbose:
        print(0, "Initial T:\t\t\t\t", T.shape)

    # First site
    mpo = [0]*n
    U, S, Vt = la.svd(T, full_matrices = False)
    U = U[:,:max_bd]
    S = S[:max_bd]
    Vt = Vt[:max_bd,:]
    mpo[0] = U.reshape((1,U.shape[0],U.shape[1]))
    T = np.dot(np.diag(S),Vt).reshape(S.size*d2,int(Vt.shape[1]/d2))
    if verbose:
        print(1, U.shape, Vt.shape, "\t->", mpo[0].shape, "\t", T.shape, "\t", S.size)

    # Interior sites
    for i in range(1,n-1):
            U, S, Vt = la.svd(T, full_matrices = False)
            U = U[:,:max_bd]
            S = S[:max_bd]
            Vt = Vt[:max_bd,:]
            mpo[i] = U.reshape((int(U.shape[0]/d2),d2,U.shape[1]))
            T = np.dot(np.diag(S),Vt).reshape((S.size*d2,int(Vt.shape[1]/d2)))
            if verbose:
                print(i+1, U.shape, Vt.shape, "\t->", mpo[i].shape, "\t", T.shape, "\t", S.size)

    # Last site
    mpo[n-1] = np.dot(np.diag(S),Vt).reshape((S.size, Vt.shape[0], 1))
    if verbose:
        print(n, "Final Matrix:\t\t  ", mpo[n-1].shape)

    return mpo

We can run this algorithm on a randomly generated density matrix. I should expect to see the following dimensions of my matrices and local tensor after performing each SVD for $d=3,n=5$: The density matrix has dimensions $3^5\times 3^5$ and the environment tensor $T$ starts out with dimensions $9\times 9^4$. After each step, we should expect $D_k=9^k$ <mark>(TODO: update this)</mark>

In [25]:
# Generate a mixed state of randomly sampled pure qudit states                                                 
# d - onsite hilbert space dimension
# N - number of sites
# M - number of pure states used
def random_mixed_state(N,d=3,M=10):
        states = [random_state(N,d) for i in range(M)]
        weights = [np.random.uniform(low=0.0,high=1.0) for i in range(M)]
        total_weight = sum(weights)
        state = np.zeros((d**N,d**N))
        for i in range(M):
                state = state + (weights[i]/total_weight) * np.outer(states[i],states[i].conj())
        return state

# Generate a random qudit state
# d - onsite hilbert space dimension
# N - number of sites
def random_state(N,d=3):
        normals = [np.random.standard_normal(size=d**N) for i in range(2)]
        state = np.array([normals[0][i] + 1j * normals[1][i] for i in range(d**N)])
        state /= la.norm(state)
        return state

In [86]:
d = 3
n = 5
rho = random_mixed_state(n,d)
m_rho = state_to_mpo(rho,n,d,verbose=True,max_bd=81)

0 Initial T:				 (9, 6561)
1 (9, 9) (9, 6561) 	-> (1, 9, 9) 	 (81, 729) 	 9
2 (81, 81) (81, 729) 	-> (9, 9, 81) 	 (729, 81) 	 81
3 (729, 81) (81, 81) 	-> (81, 9, 81) 	 (729, 9) 	 81
4 (729, 9) (9, 9) 	-> (81, 9, 9) 	 (81, 1) 	 9
5 Final Matrix:		   (9, 9, 1)


A simple check of this algorithm is to verify that $\rho_{ij}=\rho_{i_1\cdots i_n j_1\cdots j_n}=\rho_{i_1j_1\cdots i_nj_n}=\rho_{u_1\cdots u_n}=\prod_k \rho^{(k)u_k}$. Note that $u_k=d * i_k+j_k$ is the most natural mapping, but I'm not sure if that corresponds with the way reshape works (which might be $u_k = i_k + d*j_k$ or something else). Instead of doing this check for a specific index, we can reconstruct $\rho$ from the MPO $\rho^{(k)}$ by just multiplying matrices and doing the reshaping outlined above, in reverse. 

In [83]:
#TODO
def mpo_to_state(m):
    n = len(m)
    _,d,_ = m[0].shape
    
    state = np.zeros((d**n,d**n))
    tensor_shape = tuple([d]*(2*n))
    state = state.reshape(tensor_shape)
    for i in range(d**(2*n)):
        for k in range(n):
            index[k] = i/k #TODO: index properly
            temp = np.dot(temp,m[:,index[k],:])
        state[index] = temp
    
    return state

# Inner products
The Frobenius norm overlap between two operators is defined to be
$$\left<\rho|\sigma\right>=\mathrm{Tr}\left[\rho^\dagger\sigma\right]=d^{-n}\sum_u \left(\prod_i \rho^{(i)u_i}\right)^*\left(\prod_j \sigma^{(i)u_j}\right),$$
where $*$ denotes element-wise complex conjugation. This can be calculated in a linear number of computations in $d$ and $n$ and cubic in $D^\sigma$ and $D^\rho$ like so:
1. $M_{\alpha\beta} = \sum_{u\in \mathbb Z_d}(\rho^{(1)}_{1u\alpha})^*\sigma^{(1)}_{1u\beta}$
2. For $k\in\{2,\cdots,n\}:$
    - $M_{\alpha\beta}\leftarrow \sum_{u\in \mathbb Z_d,\mu\in\mathbb Z_{D^\rho_k},\mu\in\mathbb Z_{D^\sigma_k}}(\rho^{(k)}_{\mu u \alpha})^*M_{\mu \nu}\sigma^{(k)}_{\nu u \beta}$
3. Return $M_{11}$

Note that the matrix elements are stored like so: $\rho^{(1)u}_{\alpha\beta}=\rho_{\alpha u \beta}$, and $D^\rho_k$ denotes the bond dimension of $\rho$'s MPO at site $k$ 

In [90]:
# Overlap between two MPOs
# Assumptions:
#       len(m1) = len(m2)                                       [same length]
#       m1[i].shape = m2[i].shape = (bond,phys,bond)            [same bond and phys dim at each site]
#       m1[0].shape = (1,_,_) and m1[-1].shape = (_,_,1)        [closed boundary conditions]
def overlap(m1, m2):
        # Extract length, phys dim, and first site bond dim
        n = len(m1)

        # First site
        M = np.tensordot(m1[0][0].conj(),m2[0][0],axes=([0,0]))      # sum over physical indices

        #print(0,m1[0].shape,M.shape)
        # Rest of contraction
        for i in range(1,n):
                M = np.tensordot(M,m1[i].conj(),axes=([0,0]))
                M = np.tensordot(M,m2[i],axes=([0,1],[0,1]))
                #print(i,m1[i].shape, M.shape)
        return M[0][0]

Test: For the random state $\rho$, does $\mathrm{Tr}[\rho^\dagger \rho]=\left<\{\rho^{(k)}\}|\{\rho^{(k)}\}\right>$?

In [91]:
print(np.trace(np.dot(rho.T.conj(),rho)))
print(overlap(m_rho,m_rho))

(0.12005490874249065+0j)
(0.1200549087424906+8.673617379884035e-19j)


Test: Given random states $\rho$ and $\sigma$, $\mathrm{Tr}[\rho^\dagger \sigma]=\left<\{\rho^{(k)}\}|\{\sigma^{(k)}\}\right>$. Does the contraction algorithm give a similar result (and work at all) when $\{\sigma^{(k)}\}$ are generated with some singular values truncated?

In [92]:
sigma_same_bd = random_mixed_state(n,d)
m_sigma = state_to_mpo(sigma,n,d)
m_sigma_smaller = state_to_mpo(sigma,n,d,max_bd=50)
print(np.trace(np.dot(rho.T.conj(),sigma)))
print(overlap(m_rho,m_sigma))
print(overlap(m_rho,m_sigma_smaller))

(0.004206078770536007+1.8973538018496328e-19j)
(0.00420607877053601+2.439454888092385e-19j)
(0.0035503599462803634-3.709388152946339e-05j)


Test: Given a random state $\rho$ and an MPO for the identity matrix, does the overlap of the states give 1 due to normalization of $\rho$?

In [98]:
max_mixed = np.identity(d**n)
m_max = state_to_mpo(max_mixed,n,d)
print(overlap(m_rho,m_max))

(0.9999999999999999+8.732788471021416e-17j)
